In [7]:
import json
from tqdm import tqdm
import numpy as np
import os
from glob import glob

# Matches the class name and number existing in the class file
def class_dict(file_path):
    check_path = 'C:/Users/3060 ASUS1/data-conversion/종무/H-211115_A21_A_UA-05_202/Annot' # class_id is the path to the folder containing the json to be checked.
    chk_list = []

    for (path, dir, files) in os.walk(check_path):

        print(len(files))

        for fileName in files:

            if fileName.endswith('.json'):

                with open(os.path.join(path, fileName), encoding='utf-8') as json_file:

                    try:
                        jsonData = json.load(json_file)
                        LearningDataInfo = jsonData['Learning Data Info.']
                        

                        for i in range(len(LearningDataInfo['annotation'])):
                        
                            if list(LearningDataInfo['annotation'])[i]['class_id'] not in chk_list:
                                chk_list.append(list(LearningDataInfo['annotation'])[i]['class_id'])   #add every class_id name which is not in the chk_list
                            #else: # Check in che chk_list if the class_id in the chk_list => count the class_id exist in the chk_list (count["class_id"] = count["class_id"] +1)
                                
                            
                                                        


                    except Exception as ex:
                        print(ex)
                        print(fileName)
                        pass

        #print(chk_list)
        chk_list_sort = sorted(chk_list)
        #print(chk_list_sort)

    with open('C:/Users/3060 ASUS1/data-conversion/종무/H-211115_A21_A_UA-05_202/box.names.txt', 'w', encoding='utf-8') as f:
        for class_name in chk_list_sort:
            
            f.write(f'{class_name}\n')

   
    with open(file_path, encoding='utf-8') as f:
        classes = {}
        
        for i, j in enumerate(f):
            classes[j.split('\n')[0]] = i
            
    return classes

# 전체 json폴더 내에 있는 json파일 불러와서 convert
def convert_Warehouse2YOLO(input_path, output_path, class_path):
    box_class = class_dict(class_path)
    

    num = 0
    yolo_dir = open(output_path, 'a', encoding='utf-8')
    print(output_path)
    if not os.path.exists(os.path.join('/'.join(output_path.split('/')[:-1]), 'coor')):
        os.mkdir(os.path.join('/'.join(output_path.split('/')[:-1]), 'coor'))

    new_path = os.path.join('/'.join(output_path.split('/')[:-1]), 'coor')
    
    for j_file in tqdm(glob(input_path+'/*.json', recursive=True)):
        with open(j_file, encoding='utf-8') as json_file:
            json_data = json.load(json_file)
            width,height =(1920, 1080)

            # json파일 내 이미지명 추출

            img_name = json_data['Source Data Info.']['source_data_ID'] + '.jpg'

            # convert된 yolo_data_set저장 경로
            yolo_set = open('C:/Users/3060 ASUS1/data-conversion/종무/H-211115_A21_A_UA-05_202/coor'+'/'+img_name.replace('.jpg','.txt'), 'w', encoding='utf-8')

            # annotations 값 추출되는건가요?
            annotations = json_data['Learning Data Info.']['annotation']
            # file_path = j_file.replace('.json', '.jpg').replace('\\','/')
            file_path = ''

            # annotations된 객체들 반복
            for ann in annotations:
                # if ann['type'] == 'box':
                  if 'box' in ann.keys():
                    # yolo형식에 맞게 자료 변환
                    x1 = ann['box'][0]
                    y1 = ann['box'][1]
                    x2 = ann['box'][2]
                    y2 = ann['box'][3]

                    dw = 1. / width
                    dh = 1. / height
                    x = (float(x1) + float(x1) + float(x2)) / 2.0
                    y = (float(y1) + float(y1) + float(y2)) / 2.0
                    w = float(x2)
                    h = float(y2)

                    x = round(x * dw, 6)
                    w = round(w * dw, 6)
                    y = round(y * dh, 6)  # 6자리 표시
                    h = round(h * dh, 6)

                    obj = (str(box_class[ann['class_id']]) + ' '
                              + str(x) + ' '
                              + str(y) + ' '
                              + str(w) + ' '
                              + str(h)
                            )
                    # path값 설정
                    file_path += obj+'\n'

            yolo_set.write(file_path)
            yolo_set.close()
            yolo_dir.write('data/img/'+img_name+'\n')
    yolo_dir.close()


"""
    Final execution method: convert_Warehouse2YOLO in the file ('path containing json files', 'path and file name to save yolo_set', 'file name where classes are saved (.txt, .name)')
                  run as command
"""

convert_Warehouse2YOLO('C:/Users/3060 ASUS1/data-conversion/종무/H-211115_A21_A_UA-05_202/Annot', 'C:/Users/3060 ASUS1/data-conversion/종무/H-211115_A21_A_UA-05_202/train.txt', 'C:/Users/3060 ASUS1/data-conversion/종무/H-211115_A21_A_UA-05_202/box.names.txt')

100
C:/Users/3060 ASUS1/data-conversion/종무/H-211115_A21_A_UA-05_202/train.txt


100%|██████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 1663.25it/s]
